<a href="https://colab.research.google.com/github/dh610/Advanced-AI/blob/main/AirNet_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install colab_ssh --upgrade

In [3]:
from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password="1234")

In [4]:
import sys
import os

sys.path.append("/content/drive/MyDrive/Advanced-AI/AirNet/")
os.chdir("/content/drive/MyDrive/Advanced-AI/AirNet/")
pwd = "/content/drive/MyDrive/Advanced-AI/Restormer/"
!ls

airnet.yaml  cloudflared.log  data_dir	net	   README.md  test.py	utils
cloudflared  data	      demo.py	option.py  test       train.py


In [5]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import yaml
import random
import torch

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

AttributeError: module 'torch.utils._pytree' has no attribute 'register_pytree_node'

In [ ]:
data_dir = pwd + "RainTrainL/"

denoise_input_dir = data_dir + "gaussian_sigma_5/"
derain_input_dir = data_dir + "rain/"
target_dir = data_dir + "target/"


In [ ]:
class ImagePairDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(input_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        input_image_path = os.path.join(self.input_dir, self.image_filenames[idx])
        input_image = Image.open(input_image_path).convert("RGB")

        target_image_path = os.path.join(self.target_dir, self.image_filenames[idx])
        target_image = Image.open(target_image_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

denoise_dataset = ImagePairDataset(input_dir=denoise_input_dir, target_dir=target_dir, transform=transform)
derain_dataset = ImagePairDataset(input_dir=derain_input_dir, target_dir=target_dir, transform=transform)

dataset = denoise_dataset + derain_dataset

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)




In [ ]:
!pip install --target=$my_path mmcv

  Using cached mmcv-2.2.0.tar.gz (479 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.2 MB/s eta 0:00:00
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.1 MB/s eta 0:00:00
  Using cached rich-13.8.1-py3-none-any.whl.metadata (18 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached importlib_metadata-8.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached platformdirs-4.3.6-py3-none-any.whl.metadata (11 kB)
  Using cached tomli-2.0.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached zipp-3.20.2-py3-none-any.whl.metadata (3.7

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from runpy import run_path
import os
from torch.optim.lr_scheduler import CosineAnnealingLR
from net.model import AirNet

model = AirNet()
model = model.cuda()


criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-6)

ModuleNotFoundError: No module named 'mmcv'

In [ ]:
from tqdm import tqdm

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()


  0%|          | 0/90 [00:08<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 42.81 MiB is free. Process 33388 has 39.51 GiB memory in use. Of the allocated memory 38.93 GiB is allocated by PyTorch, and 91.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

model.eval()
val_loss = 0.0
with torch.no_grad():
    for data in test_loader:
        inputs = data['input'].cuda()
        targets = data['target'].cuda()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        val_loss += loss.item()
val_loss /= len(test_loader)
print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')